<h2>Machine Learning</h2>
<br>
<p>Model learning and predicting</p>

In [ ]:
#pip install catboost

In [1]:
import numpy as np
from numpy import arange
from numpy import absolute
from numpy import mean
from numpy import std
import pandas as pd
#-------sklearn etc:
import sklearn
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder ## might not scale, less effective in regression and tree based algorithms
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
#-------metrics:
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
#-------plots:
from matplotlib import pyplot as plt
#-------Extra models:
from catboost import CatBoostRegressor

<h4>Functions:</h4>

In [2]:
def split_train_test(df):
    y = df['Mean Score']
    X = df.drop('Mean Score', axis = 1)
    return train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
def r_adjust(y, y_pred, **kwargs):
    res = r2_score(y, y_pred)
    Adj_r2 = 1 - (1 - res * (len(y) - 1) / (len(y) - kwargs['X'].shape[1] - 1))
    return Adj_r2

In [4]:
def run_model(model_class, df, params):
    model = model_class(verbose=False, one_hot_max_size=255) if model_class == CatBoostRegressor else model_class()
    X_train, X_test, y_train, y_test = split_train_test(df)
    grid_search = GridSearchCV(estimator=model, param_grid=params,
                               cv=5, n_jobs=3, verbose=1,
                               scoring=make_scorer(r_adjust, greater_is_better=True, X=X_train))
    model.fit(X_train, y_train)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_predict = model.predict(X_test)
    y_predict_best = best_model.predict(X_test)
    try:
        print(f"Best hyperparams : {best_model.__dict__['_init_params']}")
    except:
        print(f"Best hyperparams: {best_model}")
    return r2_score(y_test, y_predict), r2_score(y_test, y_predict_best)

<h4>Main Body:</h4>

In [5]:
df = pd.read_csv('./clean_anime_dataframe')
df

,Format,Mean Score,Popularity,Favorites,Source,Action,Adventure,Comedy,Drama,Ecchi,...,Mecha,Music,Mystery,Psychological,Romance,Sci-Fi,Slice of Life,Sports,Supernatural,Thriller
0,0,66,6050,69,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,0,0
1,0,72,6307,98,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,60,6176,48,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
3,0,61,43462,656,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
4,2,76,58959,797,1,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9522,2,61,22,1,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9523,5,62,73,2,3,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9524,4,61,71,2,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9525,5,66,159,2,3,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


<p>First ,lets split the data to training and testing segments based on the value we want to predict 'Mean Score'</p>

In [6]:
X_train, X_test, y_train, y_test = split_train_test(df)

<p>After splitting the data lets create a linear regression model and train it based on the entire dataframe without scaling or taking out unnecessary data</p>

In [7]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [8]:
predict_val = lr_model.predict(X_test)

In [12]:
score = r_adjust(y_test, predict_val, X_test.tolist())
score

AttributeError: 'DataFrame' object has no attribute 'tolist'

<p>As you can see, the first model was unsuccessful with an accuracy of 24%, lets try and improve it</p>

In [ ]:
df['Popularity'] = np.log(df['Popularity']) 
df['Favorites'] = np.log(df['Favorites'])

In [ ]:
y = df['Mean Score']
X = df.drop('Mean Score', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size= 0.33)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

In [ ]:
predict_val = lr_model.predict(X_test)

In [ ]:
score = r2_score(y_test, predict_val)
score

<p>As you can see, using lan on the most influential columns regarding the target of the model (Mean score) helped raise the model's accuracy from 24% to 47%</p>

Although we've improved the model, I think this is as far as regular Linear regression can go in the case of this dataset.

Lets try a few more models.

First of, lets try a different variation to the Linear Regression, the Lasso Regression.

In [ ]:
lass_m = Lasso(alpha = 1.0)

In [ ]:
lass_m.fit(X_train, y_train)

In [ ]:
predicted_vals = lass_m.predict(X_test)

In [ ]:
score = r2_score(y_test, predicted_vals)
score

On first attempt the Lasso regression yielded worse result than the Linear regression, but we can still tweek it a little. contrary to Linear regression, on the Lasso we can change a hyperparameter to yield better results.

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
grid = dict()
grid['alpha'] = arange(0, 1, 0.01)

In [ ]:
search = GridSearchCV(lass_m, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

In [ ]:
results = search.fit(X_train, y_train)

In [ ]:
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

The results from the GridSearchCV shows that the best hyperparameter is 0.01 and that the mean absolute error (MAE) is -5.53, which is the average offset from the predicted value, not too shabby.

In [ ]:
lass_m = Lasso(alpha = 0.01)

In [ ]:
lass_m.fit(X_train, y_train)

In [ ]:
predicted_vals = lass_m.predict(X_test)

In [ ]:
score = r2_score(y_test, predicted_vals)
score

The result is more disappointing than expected, the Lasso with the most optimal hyperparameter gave out slightly worse results than the Linear regression with an accuracy of 47.6% compared to the 47.7% (neglible but disappointing still).

Although our previous models disappointed, I still want to try one last model that showed great potential when it comes to predicting continuous variables such as our Mean Score, the CatBoostRegressor.